## Functions / Imports

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
pd.options.mode.chained_assignment = None  # default='warn'


def str_to_int(x):
    '''Transform class4 to numeric labels.'''
    if x=="nonevent":
        return 0 
    elif x=="Ia":
        return 1
    elif x=="Ib":
        return 2
    else:
        return 3


def int_to_str(x):
    '''Transform class 4 back to events.'''
    if x==0:
        return "nonevent"
    elif x==1:
        return "Ia"
    elif x==2:
        return "Ib"
    else:
        return "II"
    

def read_data(path="data/npf_train.csv"):
    '''Read train/test data. Returns filtered out normalized data.'''
    data = pd.read_csv(path)                                                        # read the data
    class4 = list(map(lambda x: int(str_to_int(x)), data["class4"]))                # encode events into integers
    data = data.drop(columns=["id", "partlybad", "date", "class4"])                 # drop unnecessary columns
    data = data.iloc[:,[0,10,22,24,36,48,58,60,62,64,66,78,80,82,84,94,96,98]]      # choose important variables
    data = (data-data.mean()) / data.std()                                          # normalize the data
    data["class4"] = class4
    
    return data


def report_accuracies(y_pred, y_test, verbose=True):
    '''Calculate each class accuracy. Return multi class and binary accuracies.'''
    acc_0 = 0; acc_1 = 0; acc_2 = 0; acc_3 = 0 
    for i in range(len(y_pred)):
        if y_test[i] == 0 and y_pred[i] == 0:
            acc_0 += 1
        elif y_test[i] == 1 and y_pred[i] == 1:
            acc_1 += 1
        elif y_test[i] == 2 and y_pred[i] == 2:
            acc_2 += 1
        elif y_test[i] == 3 and y_pred[i] == 3:
            acc_3 += 1
            
    acc0 = acc_0 / (np.mean(y_test == 0)*len(y_test))
    acc1 = acc_1 / (np.mean(y_test == 1)*len(y_test))
    acc2 = acc_2 / (np.mean(y_test == 2)*len(y_test))
    acc3 = acc_3 / (np.mean(y_test == 3)*len(y_test))
    acc = np.mean(y_test==y_pred)
    
    if verbose:
        print(f"Accuracy for nonevent = {acc0:.4f} \t({acc_0}/{int((np.mean(y_test == 0)*len(y_test)))})")
        print(f"Accuracy for event Ia = {acc1:.4f} \t({acc_1}/{int((np.mean(y_test == 1)*len(y_test)))})")
        print(f"Accuracy for event Ib = {acc2:.4f} \t({acc_2}/{int((np.mean(y_test == 2)*len(y_test)))})")
        print(f"Accuracy for event II = {acc3:.4f} \t({acc_3}/{int((np.mean(y_test == 3)*len(y_test)))})")
        print(f"Accuracy multi-class = {acc:.4f}")

    # calculate binary accuracy
    for i, y in enumerate(y_pred):
        if y != 0:
            y_pred[i] = 1
        if y_test[i] != 0:
            y_test[i] = 1
            
    bin_acc = np.mean(y_pred == y_test)
    
    return acc, bin_acc


def split_train_test(data, a=0.5):
    '''Split each class evenly.'''
    data = data.sample(frac=1)
    X_nonevent, y_nonevent = data.loc[data["class4"] == 0].drop(columns=["class4"]).values, data["class4"].loc[data["class4"] == 0].values
    X_Ia, y_Ia = data.loc[data["class4"] == 1].drop(columns=["class4"]).values, data["class4"].loc[data["class4"] == 1].values
    X_Ib, y_Ib = data.loc[data["class4"] == 2].drop(columns=["class4"]).values, data["class4"].loc[data["class4"] == 2].values
    X_II, y_II = data.loc[data["class4"] == 3].drop(columns=["class4"]).values, data["class4"].loc[data["class4"] == 3].values

    # Split each class into equal size train and test
    X_nonevent_train, X_nonevent_test, y_nonevent_train, y_nonevent_test = train_test_split(X_nonevent, y_nonevent, test_size=a)
    X_Ia_train, X_Ia_test, y_Ia_train, y_Ia_test = train_test_split(X_Ia, y_Ia, test_size=a)
    X_Ib_train, X_Ib_test, y_Ib_train, y_Ib_test = train_test_split(X_Ib, y_Ib, test_size=a)
    X_II_train, X_II_test, y_II_train, y_II_test = train_test_split(X_II, y_II, test_size=a)

    # Recreate the mixed class train and test data (full data)
    X_train = np.vstack([X_nonevent_train, X_Ia_train, X_Ib_train, X_II_train])
    X_test = np.vstack([X_nonevent_test, X_Ia_test, X_Ib_test, X_II_test])
    y_train = np.hstack([y_nonevent_train, y_Ia_train, y_Ib_train, y_II_train])
    y_test = np.hstack([y_nonevent_test, y_Ia_test, y_Ib_test, y_II_test])
    
    return X_train, X_test, y_train, y_test

## Read data

In [4]:
data = read_data()
data.shape

(458, 19)

## Prepare X, y and fit SVM

In [5]:
X, y = data.drop(columns="class4").values, data.class4.values

svm = SVC(kernel='rbf', C=3.75, tol=1e-8)
scores = cross_val_score(svm, X, y, cv=5)
print(f"Mean accuracy (5-fold CV): {np.mean(scores):.4f}")

Mean accuracy (5-fold CV): 0.6661


## Performance on hidden test

In [20]:
train_data = read_data()
test_data = read_data(path="data/npf_test.csv")

X_train, y_train = train_data.drop(columns="class4").values, data.class4.values
X_test, y_test = test_data.drop(columns="class4").values, test_data.class4.values

clf = SVC(kernel='rbf', C=3.75, tol=1e-8)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
multi, binary = report_accuracies(y_pred, y_test)
print(f"Accuracy binary = {binary:.4f}")

Accuracy for nonevent = 0.9332 	(475/509)
Accuracy for event Ia = 0.1458 	(7/48)
Accuracy for event Ib = 0.5000 	(84/168)
Accuracy for event II = 0.6042 	(145/240)
Accuracy multi-class = 0.7368
Accuracy binary = 0.9016


### Performance on hidden test using high $C$ value (completely overfitting)

In [21]:
train_data = read_data()
test_data = read_data(path="data/npf_test.csv")

X_train, y_train = train_data.drop(columns="class4").values, data.class4.values
X_test, y_test = test_data.drop(columns="class4").values, test_data.class4.values

clf = SVC(kernel='rbf', C=1000, tol=1e-8)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
multi, binary = report_accuracies(y_pred, y_test)
print(f"Accuracy binary = {binary:.4f}")

Accuracy for nonevent = 0.9136 	(465/509)
Accuracy for event Ia = 0.5625 	(27/48)
Accuracy for event Ib = 0.6071 	(102/168)
Accuracy for event II = 0.6083 	(146/240)
Accuracy multi-class = 0.7668
Accuracy binary = 0.9109


## Fit 10000 SVMs 

In [8]:
def cv(estimator, data, k=100, a=0.5):
    '''k: number of SVMs trained
       a: test_size when splitting into train/test.'''
    scores = []
    bin_scores = []
    
    for _ in range(k):
        X_train, X_test, y_train, y_test = split_train_test(data, a=a)
        
        estimator.fit(X_train, y_train)
        y_pred = estimator.predict(X_test)
        
        acc, acc0 = report_accuracies(y_pred, y_test, verbose=0)
        scores.append(acc)
        bin_scores.append(acc0)
    
    return scores, bin_scores

data = read_data()
data = data.drop(columns=["PAR.mean", "UV_B.mean", "NET.mean"])
clf = SVC(C=3.75, kernel="rbf", tol=1e-5)

k=10000
scores, bin_scores = cv(clf, data, k)
print(f"Training {k} SVMs...")
print(f"Average accuracy = {np.mean(scores):.4f}")
print(f"Maximum accuracy = {np.max(scores):.4f}")
print(f"Average accuracy for binary task = {np.mean(bin_scores):.4f}")
print(f"Maximum accuracy for binary task = {np.max(bin_scores):.4f}")

Training 10000 SVMs...
Average accuracy = 0.6556
Maximum accuracy = 0.7359
Average accuracy for binary task = 0.8452
Maximum accuracy for binary task = 0.9134


## Calculate probabilities and labels for answers.csv

In [10]:
train_data = read_data()
test_data = read_data(path="data/npf_test.csv")

X_train, y_train = train_data.drop(columns="class4").values, data.class4.values
X_test, y_test = test_data.drop(columns="class4").values, test_data.class4.values

clf = SVC(C=3.75, kernel="rbf", tol=1e-5, probability=True)
clf.fit(X_train, y_train)
predicted_probabilities = clf.predict_proba(X_test)

labels = []
p = []
for row in predicted_probabilities:
    prob = np.max(row)
    label = int(np.argmax(row))
    label = int_to_str(label)
    if label=="nonevent":
        prob = 1-prob
    prob=round(prob,2)
    labels.append(label)
    p.append(prob)

answers = pd.DataFrame(labels, columns=["class4"])
answers["p"] = p

## Write to answers.csv

In [ ]:
# Write answers
with open("check/answers.csv", "w") as f:
    f.write("0.87\n")
    answers.to_csv(f, index=False, line_terminator='\n')

## Sanity checks

In [12]:
answers = pd.read_csv("check/answers.csv", header=1)
answers["class2"] = "event"
answers["class2"].loc[answers.class4 == "nonevent"] = "nonevent"
answers.head()

,class4,p,class2
0,nonevent,0.20,nonevent
1,Ib,0.46,event
2,nonevent,0.11,nonevent
3,II,0.49,event
4,II,0.52,event


In [13]:
test_data = read_data(path="data/npf_test.csv")
test_data["class2"] = "event"
test_data["class4"] = list(map(lambda x: int_to_str(x), test_data["class4"]))
test_data["class2"].loc[test_data.class4 == "nonevent"] = "nonevent"
test_data.loc[:,["class4","class2"]].head()

,class4,class2
0,II,event
1,Ib,event
2,nonevent,nonevent
3,Ib,event
4,II,event


## See where predicted and true labels don't match

In [14]:
answers["class4_npf"] = test_data.class4
answers["class2_npf"] = test_data.class2
answers.loc[answers.class2 != test_data.class2].head(20)

,class4,p,class2,class4_npf,class2_npf
0,nonevent,0.20,nonevent,II,event
6,nonevent,0.39,nonevent,II,event
9,II,0.67,event,nonevent,nonevent
19,II,0.61,event,nonevent,nonevent
26,nonevent,0.17,nonevent,II,event
36,nonevent,0.59,nonevent,II,event
39,nonevent,0.46,nonevent,Ia,event
41,nonevent,0.39,nonevent,II,event
73,nonevent,0.49,nonevent,II,event
87,nonevent,0.60,nonevent,Ia,event
